In [1]:
%load_ext Cython

In [2]:
%%cython
from cython.view cimport array as cvarray
import numpy as np
cimport numpy as np
from numpy cimport ndarray
from libc.math cimport log, sqrt, fmax, ceil, floor, trunc
ctypedef np.float64_t dtype_t
ctypedef np.int32_t dtypei_t

cdef int sgn(float fl):
    return (fl > 0) - (fl < 0)

cdef int intsgn(double inte):
    return (inte > 0) - (inte < 0)

def  deeDeeEm(float v = 0, float a = 1, float w = 0.5, int n_samples = 20000, float max_t = 4.00000000, float dt = 0.05, int binned = 0, float bandwidth = 0.05):
    
    cdef np.ndarray[float, ndim=1] reactionTime = np.zeros(n_samples, dtype = np.float32)
    cdef np.ndarray[int, ndim=1] reactionDir  = np.zeros(n_samples, dtype = np.int32)
    cdef float y
    cdef float t
    for i in range(n_samples):
        y = -a + (2 * w * a)
        t = 0.0
        while(y > -a and y < a and t < max_t):
            y += v*dt + np.sqrt(dt)*np.random.normal()
            t += dt
        reactionTime[i] = t
        reactionDir[i] = -1 * sgn(y)
    if binned == 1:
        return binnedBoys(timeData = reactionTime, choice = reactionDir)
        
    else:
        return (reactionTime,reactionDir)
        

def binnedBoys(ndarray[float, ndim =1] timeData, ndarray[int, ndim =1] choice, float binwidth = 0.05, float max_t = 4.000000):
    cdef double temp
    cdef int numToInt = 0
    cdef int binNum = <int>ceil(max_t/binwidth) + 1
    cdef np.ndarray[int, ndim=2] binCount = np.zeros((binNum, 2), dtype = np.int32) #form the array that stores the result
    for i in range(len(timeData)):
        numToInt = 0
        temp = 0
        temp = floor(timeData[i]/binwidth)
        #print(temp)
        numToInt = <int>temp
        if choice[i] == -1:
            binCount[numToInt][0] +=1
        else:
            binCount[numToInt][1] +=1
    #for i in binCount:
        #print(i)  
        
    return binCount





In [3]:
import scipy.optimize as op
import math
import numpy as np
#original dataset

def paramProducer(v, a, w, n_samples, max_t, dt):
    return deeDeeEm(v, a, w, n_samples, max_t, dt, binned = 1)
    
param = (paramProducer(0, 0.25, 0.25, 20000, 4.000000, 0.05))
    

def f(x, *params):
    v, a, w = x
    result = deeDeeEm(v, a, w, binned = 1)
    binCount = 20000
    print(x)
    return np.sum(-(np.multiply(np.log((result+1)/20000), param)))

rranges = (slice(-2, 2, (4/10)), slice(.25, 2, (2.5/10)), slice(.25, .8, (1/10)))

def optimizer(function, rangeD):
    bruteforce = op.brute(function, rangeD, finish = None, workers = -1)
    return bruteforce
#function to be optimised is log(p1^n1 p2^n2 p3^ n3... pn ^nn)



In [4]:
optimizer(f, rranges)

[-2.    0.25  0.25]
[-1.6   0.25  0.25]
[-1.2   0.25  0.25]
[-2.    0.25  0.35]
[-1.6   0.25  0.35]
[-1.2   0.25  0.35]
[-2.    0.75  0.45]
[-1.6   0.25  0.45]
[-2.    0.25  0.45]
[-1.6   0.75  0.45]
[-1.2   0.25  0.45]
[-1.2   0.75  0.45]
[-1.6   0.25  0.55]
[-2.    0.25  0.55]
[-1.2   0.25  0.55]
[-1.6   0.25  0.65]
[-2.    0.25  0.65]
[-1.2   0.25  0.65]
[-2.    1.25  0.65]
[-2.    0.75  0.55]
[-1.6   0.75  0.55]
[-1.6   1.25  0.65]
[-1.6   0.25  0.75]
[-2.    0.25  0.75]
[-1.2   0.25  0.75]
[-1.2   0.75  0.55]
[-2.    0.5   0.25]
[-1.6   0.5   0.25]
[-1.2   0.5   0.25]
[-2.    0.75  0.65]
[-2.    0.5   0.35]
[-1.6   0.5   0.35]
[-1.6   0.75  0.65]
[-1.2   0.5   0.35]
[-2.    0.5   0.45]
[-1.2   0.75  0.65]
[-1.6   0.5   0.45]
[-2.    1.25  0.75]
[-2.    0.75  0.75]
[-1.2   0.5   0.45]
[-1.6   1.25  0.75]
[-2.    0.5   0.55]
[-1.6   0.75  0.75]
[-1.6   0.5   0.55]
[-2.    1.    0.25]
[-2.    1.5   0.25]
[-1.2   0.5   0.55]
[-2.    0.5   0.65]
[-1.2   0.75  0.75]
[-1.6   1.    0.25]


array([0.  , 0.25, 0.25])

In [16]:
rranges = (slice(-4, 4, 0.25), slice(-4, 4, 0.25))
rranges

-2 -> 2 for v
.5 -> 2     a
.2 -> .8    w

(slice(-4, 4, 0.25), slice(-4, 4, 0.25))

In [24]:
%%cython
from libc.stdlib cimport rand, RAND_MAX
from libc.math cimport log, sqrt, fmax

import numpy as np
import pandas as pd
from time import time
import inspect

DTYPE = np.float32

# Method to draw random samples from a gaussian
cdef float random_uniform():
    cdef float r = rand()
    return r / RAND_MAX

cdef float random_gaussian():
    cdef float x1, x2, w
    w = 2.0

    while(w >= 1.0):
        x1 = 2.0 * random_uniform() - 1.0
        x2 = 2.0 * random_uniform() - 1.0
        w = x1 * x1 + x2 * x2

    w = ((-2.0 * log(w)) / w) ** 0.5
    return x1 * w

cdef int sign(float x):
    return (x > 0) - (x < 0)

cdef float csum(float[:] x):
    cdef int i
    cdef int n = x.shape[0]
    cdef float total = 0
    
    for i in range(n):
        total += x[i]
    
    return total

cdef void assign_random_gaussian_pair(float[:] out, int assign_ix):
    cdef float x1, x2, w
    w = 2.0

    while(w >= 1.0):
        x1 = 2.0 * random_uniform() - 1.0
        x2 = 2.0 * random_uniform() - 1.0
        w = x1 * x1 + x2 * x2

    w = ((-2.0 * log(w)) / w) ** 0.5
    out[assign_ix] = x1 * w
    out[assign_ix + 1] = x2 * 2

cdef float[:] draw_gaussian(int n):
    # Draws standard normal variables - need to have the variance rescaled
    cdef int i
    cdef float[:] result = np.zeros(n, dtype=DTYPE)
    for i in range(n // 2):
        assign_random_gaussian_pair(result, i * 2)
    if n % 2 == 1:
        result[n - 1] = random_gaussian()
    return result

# Simulate (rt, choice) tuples from: SIMPLE DDM -----------------------------------------------

# Simplest algorithm
def ddm(float v = 0, # drift by timestep 'delta_t'
        float a = 1, # boundary separation
        float w = 0.5,  # between 0 and 1
        float s = 1, # noise sigma
        float delta_t = 0.001, # timesteps fraction of seconds
        float max_t = 20, # maximum rt allowed
        int n_samples = 20000, # number of samples considered
        print_info = True # timesteps fraction of seconds
        ):

    rts = np.zeros((n_samples, 1), dtype=DTYPE)
    choices = np.zeros((n_samples, 1), dtype=np.intc)
    cdef float[:, :] rts_view = rts
    cdef int[:, :] choices_view = choices

    cdef float delta_t_sqrt = sqrt(delta_t)
    cdef float sqrt_st = delta_t_sqrt * s

    cdef float y, t

    cdef int n
    cdef int m = 0
    cdef int num_draws = int(max_t / delta_t + 1)
    cdef float[:] gaussian_values = draw_gaussian(num_draws)

    # Loop over samples
    for n in range(n_samples):
        y = w * a # reset starting point
        t = 0.0 # reset time

        # Random walker
        while y <= a and y >= 0 and t <= max_t:
            y += v * delta_t + sqrt_st * gaussian_values[m] # update particle position
            t += delta_t
            m += 1
            if m == num_draws:
                gaussian_values = draw_gaussian(num_draws)
                m = 0

        # Note that for purposes of consistency with Navarro and Fuss, 
        # the choice corresponding the lower barrier is +1, higher barrier is -1
        rts_view[n, 0] = t # store rt
        choices_view[n, 0] = (-1) * sign(y) # store choice
        
    return (rts, choices, {'v': v,
                           'a': a,
                           'w': w,
                           's': s,
                           'delta_t': delta_t,
                           'max_t': max_t,
                           'n_samples': n_samples,
                           'simulator': 'ddm',
                           'boundary_fun_type': 'constant',
                           'possible_choices': [-1, 1]})


In [60]:
import math
import numpy as np
#original dataset

def paramProducer(v, a, w, n_samples, max_t, dt):
    return deeDeeEm(v, a, w, n_samples, max_t, dt, binned = 1)
    
param = (paramProducer(0, 0.25, 0.25, 20000, 4.000000, 0.05))
    

def f(x, *params):
    v, a, w = x
    result = deeDeeEm(v, a, w, binned = 1)
    binCount = 20000
    print(x)
    return np.sum(-(np.multiply(np.log((result+1)/20000), param)))

rranges = (slice(-2, 2, (4/10)), slice(.25, 2, (2.5/10)), slice(.25, .8, (1/10)))

def optimizer(function, rangeD):
    bruteforce = op.brute(function, rangeD, finish = None, workers = -1)
    return bruteforce

strat2 = 'rand1exp'

def optimizer2(function, rangeD, strat):
    strat1 = 'best1exp'
    strat2 = 'rand1exp'
    evo = op.differential_evolution(function, rangeD, strat2, popsize = 25, disp = True, init = 'latinhypercube', polish = True, workers = -1)
    return evo


evoRange = ((-2, 2), (.25, 2), (.1, .8))

In [61]:
optimizer2(f, evoRange, strat2)

[-0.07862158  0.37144657  0.45831527]
[1.79899102 0.64891532 0.6108585 ]
[-0.54154452  0.63049357  0.53200813]
[1.11894597 0.3166846  0.41455044]
[-1.65899632  1.81994071  0.37116214]
[-1.40254186  1.12886483  0.40178076]
[1.96517859 1.41317604 0.34242499]
[-1.80753178  1.77080292  0.24254516]
[0.6628445  0.51316815 0.54048735]
[-0.41195013  1.21873273  0.68016592]
[-0.12829144  0.68835498  0.71992615]
[1.90267952 0.55674188 0.71045398]
[-0.0193876   0.73967961  0.52530239]
[-1.35972408  1.72961403  0.76856264]
[0.0410607  1.36866206 0.16013565]
[1.86440841 0.48771055 0.38391741]
[-0.79192055  0.39352704  0.15585673]
[-1.74279696  0.60226946  0.51827288]
[0.54888087 0.59847289 0.79215239]
[-1.86717304  0.94441463  0.36979112]
[-0.62655806  0.35225287  0.76101155]
[-0.16257108  1.89161773  0.43945276]
[1.6049957  1.99863412 0.33100909]
[-0.31219664  1.44770136  0.75249935]
[1.27195348 1.32807013 0.21966889]
[0.72155635 0.79150548 0.26739583]
[-0.66965864  1.76332574  0.28877587]
[-1.074

differential_evolution step 2: f(x)= 50915.8
[1.05291899 0.29664919 0.55951432]
[1.76945213 0.31330694 0.31428635]
[1.10665259 0.37144657 0.45831527]
[-1.2677363   0.48082148  0.37116214]
[-1.54208058  0.62668903  0.38078933]
[0.90031346 0.27919594 0.36415323]
[0.92882515 0.63030145 0.14460061]
[1.65424521 0.3766157  0.11723935]
[0.23208237 0.36719041 0.79407902]
[0.64812522 0.25345986 0.11673961]
[1.41930903 0.96308347 0.67006737]
[-1.59463881  0.62033001  0.22403027]
[-0.69415485  0.35698936  0.75185848]
[-0.45866079  1.12886483  0.20568211]
[1.34796107 0.60226946 0.20940726]
[0.20481533 0.35111084 0.15886116]
[1.09626439 0.90470926 0.14805499]
[1.06839583 1.72961403 0.52848459]
[-0.60999919  0.6179361   0.3028089 ]
[-0.31028487  0.52063445  0.36653231]
[-0.7477931   1.70796948  0.2759189 ]
[0.69330932 1.45483905 0.18712649]
[-1.05523114  0.50794544  0.56895338]
[0.91301528 0.41821396 0.23585797]
[-0.53558074  0.4493007   0.40999275]
[-0.47741369  0.37479437  0.10811503]
[0.39053389 

[1.53263395 0.26097298 0.28654831]
[0.21650323 1.42787744 0.16398554]
differential_evolution step 5: f(x)= 49516
[0.43501915 0.31330694 0.35753669]
[0.78939014 0.33557834 0.35723194]
[1.75876539 0.35024558 0.2897382 ]
[-0.58767056  0.5074023   0.12605601]
[-0.49851428  0.4115696   0.24149796]
[-0.78198242  0.62084374  0.171969  ]
[-0.41813783  0.28238522  0.19307686]
[0.33977205 0.3227096  0.63449563]
[-0.4324615   0.25315401  0.30081663]
[-0.34407253  0.62633252  0.39408268]
[1.44107812 0.41894165 0.19159809]
[-0.89886566  0.29138826  0.1421998 ]
[0.81614079 0.44011647 0.46664515]
[-0.53691345  0.30523805  0.17397643]
[0.36754544 0.29883418 0.54391064]
[0.78189976 0.3766157  0.4054864 ]
[-0.8450535   0.43944884  0.20955268]
[1.86645992 0.53403309 0.21081319]
[-0.46984711  0.96394599  0.2181321 ]
[-0.18223164  0.31078933  0.42906344]
[1.67537424 0.60880311 0.27594305]
[-1.13586308  0.62033001  0.52140592]
[1.2613834  1.56357178 0.45831527]
[1.23758186 0.83359538 0.40952069]
[0.18109343

[-0.14189798  0.2937116   0.34969604]
[-0.89682464  0.86595603  0.31229573]
[0.41780234 0.89402487 0.10788587]
[-0.11224915  1.20905569  0.52347116]
differential_evolution step 8: f(x)= 49516
[-0.21802358  0.25431983  0.18484461]
[0.92882515 0.2690207  0.42936201]
[1.68933535 0.32451235 0.1842435 ]
[0.97944285 0.5074023  0.27487763]
[-0.11741821  0.27820136  0.39127859]
[-0.28618372  0.54624565  0.12863062]
[0.15440018 0.62084374 0.36242517]
[0.23208237 0.30687773 0.2598783 ]
[0.33321043 0.29179159 0.32198405]
[-0.4324615   0.26012204  0.22437239]
[-0.6797593   0.26283459  0.34343663]
[-0.64048732  0.93695168  0.3557107 ]
[0.29182262 0.29883418 0.35340761]
[-0.75098145  0.27348829  0.26386736]
[-0.56294026  0.55436443  0.29265185]
[-1.44120652  0.37705804  0.2439831 ]
[0.05957069 0.25494293 0.15886116]
[0.72772842 0.3018819  0.31466116]
[-1.24180341  1.77136604  0.11723935]
[0.00850825 0.28143516 0.1981261 ]
[0.72278482 0.25071521 0.16115472]
[0.59918867 0.33163953 0.30076568]
[0.83822

[0.00561325 0.87150471 0.26263114]
[-0.33210875  0.25057154  0.17595065]
[-0.27439169  0.35220345  0.26387271]
[0.41512925 0.2797338  0.3041039 ]
[-0.287661    1.68338529  0.25366587]
[0.18001254 1.82792977 0.33565791]
[-0.38276642  0.28380845  0.38540463]
differential_evolution step 11: f(x)= 49489.5
[0.40810817 0.25756908 0.14460061]
[-1.04114595  0.26088626  0.39065205]
[0.04012215 0.25154303 0.32812774]
[0.97944285 0.26929446 0.18667386]
[-0.34407253  0.28863955  0.39833901]
[-0.14039068  0.35910032  0.24316708]
[-0.00911353  0.46296624  0.28014818]
[-0.54466881  0.26044461  0.30657838]
[-0.28632085  0.307945    0.22721829]
[-0.35135216  0.29359952  0.24370606]
[0.65600574 0.25711083 0.3343458 ]
[-0.51142048  0.25148122  0.26580163]
[-0.62069662  0.29718699  0.1421998 ]
[-0.58656642  0.26990948  0.25220249]
[-0.41334417  0.81320245  0.1826754 ]
[0.8543839  0.25258635 0.28681557]
[-0.90950623  0.26537071  0.26737327]
[0.21003066 0.33174243 0.26682432]
[0.3543171  0.25839852 0.346111

[0.09378331 1.29033293 0.3028358 ]
[0.22351393 0.27886223 0.24929624]
[-0.07414069  0.25554499  0.24394363]
[-0.33886088  0.62270506  0.27237   ]
[-0.25902297  0.27318867  0.22700247]
[-0.07209989  0.43691816  0.31280654]
[0.07339558 0.25076035 0.24790559]
[-0.10753359  1.79260952  0.23418901]
[-0.25121567  0.25234613  0.36476939]
[0.02916765 1.02456829 0.25063889]
[0.62266315 1.44310405 0.26754646]
differential_evolution step 14: f(x)= 49471.1
[0.07339558 0.25076035 0.24790559]
[0.07339559 0.25076035 0.24790559]
[0.07339558 0.25076036 0.24790559]
[0.07339558 0.25076035 0.2479056 ]
[2.  2.  0.8]
[2.00000001 2.         0.8       ]
[2.         2.00000001 0.8       ]
[2.         2.         0.80000001]
[0.08632653 0.26250088 0.25161113]
[0.08632654 0.26250088 0.25161113]
[0.08632653 0.26250089 0.25161113]
[0.08632653 0.26250088 0.25161114]
[0.07516385 0.25236585 0.24841231]
[0.07516386 0.25236585 0.24841231]
[0.07516385 0.25236586 0.24841231]
[0.07516385 0.25236585 0.24841232]
[0.07357725 

     fun: 49471.11010244374
 message: 'Optimization terminated successfully.'
    nfev: 1189
     nit: 14
 success: True
       x: array([0.07339558, 0.25076035, 0.24790559])

In [34]:
/*Run1: Default

fun: 49672.38577980804
 message: 'Optimization terminated successfully.'
    nfev: 686
     nit: 13
 success: True
       x: array([0.00103038, 0.25324416, 0.26651996])
            
Run 2: Init hypercube, Polished
      fun: 49718.23134564015
 message: 'Optimization terminated successfully.'
    nfev: 742
     nit: 13
 success: True
       x: array([0.04552492, 0.25194754, 0.23135205])

Run 3: Same as Run 2
    
    fun: 49718.390005914174
 message: 'Optimization terminated successfully.'
    nfev: 669
     nit: 12
 success: True
       x: array([0.0428955 , 0.25331897, 0.23111835])
            
Run 4: Same as 2, 3
     fun: 49735.79842667859
 message: 'Optimization terminated successfully.'
    nfev: 702
     nit: 13
 success: True
       x: array([0.17338108, 0.25612189, 0.21437459])
            
Run 5: pop size = 20
     fun: 49679.85404115095
 message: 'Optimization terminated successfully.'
    nfev: 900
     nit: 13
 success: True
       x: array([-0.07191496,  0.25216652,  0.27264885])

Run 6: pop size = 20
    
     fun: 49228.663771603635
 message: 'Optimization terminated successfully.'
    nfev: 1118
     nit: 13
 success: True
       x: array([-0.01093163,  0.25292002,  0.23396123])


Run 7: pop size = 15, strat = best1exp

    fun: 49500.202746627656
 message: 'Optimization terminated successfully.'
    nfev: 1114
     nit: 13
 success: True
       x: array([0.01024221, 0.25067603, 0.24662018])
            
Run 8: 
    
        fun: 49471.11010244374
 message: 'Optimization terminated successfully.'
    nfev: 1189
     nit: 14
 success: True
       x: array([0.07339558, 0.25076035, 0.24790559])
    
    

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 12)

In [66]:
%%cython

from libc.math cimport log, sqrt, fmax, ceil, floor, trunc

print(floor(3.99999))

3.0


In [83]:
import matplotlib.pyplot as plt